In [1]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm

In [2]:
# 컬럼 상수
user_id = 'user_id'
base_date = 'base_date'
route_id = 'route_id'
route_name = 'route_name'
route_no = 'route_no'
geton_datetime = 'geton_datetime'
geton_station_id = 'geton_station_id'
geton_station_name = 'geton_stataion_name'
geton_station_longitude = 'geton_station_longitude'
geton_station_latitude = 'geton_station_latitude'
getoff_datetime = 'getoff_datetime'
getoff_station_id = 'getoff_station_id'
getoff_station_name = 'getoff_station_name'
getoff_station_longitude = 'getoff_station_longitude'
getoff_station_latitude = 'getoff_station_latitude'
user_type = 'user_type'
user_count = 'user_count'
input_date = 'input_date'
usage = 'usage' # not excel column, for new DataFrame object
key = 'key'

In [3]:
def make_path(date):
    root_path = "C:/tb_bus_user_usage"
    base_name = "tb_bus_user_usage_"
    extender = ".csv"
    
    date = str(date)
    y = date[2:4]
    m = date[5:7]
    d = date[8:10]
    
    file_name = root_path+"/"+base_name+y+m+d+extender
    return file_name

In [4]:
def analyze_how_many_is_station_used(file_path):
    df = pd.read_csv(file_path, encoding='utf-8')

    df[geton_station_name] = df[geton_station_name].apply(lambda x : str(x))
    df[geton_station_longitude] = df[geton_station_longitude].apply(lambda x : str(x))
    df[geton_station_latitude] = df[geton_station_latitude].apply(lambda x : str(x))
    df[key] = df[geton_station_name]+"@"+df[geton_station_longitude]+"@"+df[geton_station_latitude]

    df = pd.DataFrame(df[key].value_counts())
    df.index.name = key
    df = df.rename(columns={key:usage})
    return df

def analyze_how_many_did_user_use_station(file_path):
    df = pd.read_csv(file_path, encoding='utf-8')
    
    df[geton_station_name] = df[geton_station_name].apply(lambda x : str(x))
    df[geton_station_longitude] = df[geton_station_longitude].apply(lambda x : str(x))
    df[geton_station_latitude] = df[geton_station_latitude].apply(lambda x : str(x))
    df[key] = df[geton_station_name]+"@"+df[geton_station_longitude]+"@"+df[geton_station_latitude]
    
    df = df[[key, user_count]].groupby(key).sum()
    df.index.name = key
    df = df.rename(columns={user_count:usage})
    return df

In [5]:
def combine_result(df_a, df_b):
    df_a_ = pd.DataFrame(index = set(df_b.index)-set(df_a.index))
    df_b_ = pd.DataFrame(index = set(df_a.index)-set(df_b.index))

    df_a_[usage] = 0
    df_b_[usage] = 0
    
    df_a = pd.concat([df_a, df_a_])
    df_b = pd.concat([df_b, df_b_])
    
    return df_a.add(df_b)

In [6]:
def analyze_total_usage(start_date, end_date, scale = "station"):# scale = ["station" | "user"]
    if(scale == "station"):# select analyze function and scale column
        scale = geton_station_name
        analyze_func = analyze_how_many_is_station_used
    elif(scale == "user"):
        scale = user_count
        analyze_func = analyze_how_many_did_user_use_station
        
    file_path = make_path(start_date)
    result = pd.DataFrame(analyze_func(file_path))

    for day in tqdm(range((end_date-start_date).days)):
        date = start_date + datetime.timedelta(days = day+1)
        file_path = make_path(date)
        result = combine_result(result, analyze_func(file_path))
    
    
    
    columns = [geton_station_name, geton_station_longitude, geton_station_latitude]
    result = result.reset_index().rename(columns = {"index":key})
    
    for i, column in enumerate(columns):
        result[column] = result[key].apply(lambda x : str(x).split("@")[i])
    
    columns.append(usage)
    result = result[columns]
    result = result.dropna(axis = 0)
    result = result[(result[geton_station_longitude] != "nan") & (result[geton_station_latitude] != "nan")]
    result = result.sort_values(by=usage, ascending = False)
    return result

In [7]:
def make_new_path(start_date, end_date):
    s_d = str(start_date)
    e_d = str(end_date)

    s_y, s_m, s_d = s_d[2:4], s_d[5:7], s_d[8:10]
    e_y, e_m, e_d = e_d[2:4], e_d[5:7], e_d[8:10]
    
    return "tag_usage_data_"+s_y+s_m+s_d+"_"+e_y+e_m+e_d+".csv"

In [21]:
import xml.etree.ElementTree as elemTree
import requests
def get_address(loc_x, loc_y, min_x = 126.531891, min_y = 33.399409, key = "E20F6493-C13D-3F6F-AC90-D5BB2F239901"):
    loc_x = round(float(loc_x), 7)
    loc_y = round(float(loc_y), 7)
    url_form = "http://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point={},{}&format=xml&type=both&zipcode=true&simple=false&key={}"
    url = url_form.format(loc_x, loc_y, key)
    response = requests.get(url)
    tree = elemTree.fromstring(response.text)
    branch = ""
    try:
#     road = tree.find("result").find("item[2]").find("text").text
        branch = tree.find("result").find("item[1]").find("text").text
    except:
        print(loc_x, loc_y)
        x_left = str(int(loc_x))
        y_left = str(int(loc_y))
        
        
        if min_x < loc_x:
            x_right = str((int(loc_x*100000)-2)%100000)
        else:
            x_right = str((int(loc_x*100000)+2)%100000)
            
        if min_y < loc_y:
            y_right = str((int(loc_y*100000)-2)%100000)
        else:
            y_right = str((int(loc_y*100000)+2)%100000)
            
        loc_x = float(x_left+"."+x_right)
        loc_y = float(y_left+"."+y_right)
        return get_address(loc_x, loc_y)
            
    return branch

# loc_x = 126.30207
# loc_y = 33.2253
# print(get_address(loc_x, loc_y))

In [23]:
def set_address_column(df):
    df = pd.DataFrame.copy(df)
    df["address"] = ""
    for i in tqdm(df.index):
        df.loc[i, "address"] = get_address(float(df.loc[i, "LOCAL_X"]), float(df.loc[i, "LOCAL_Y"]))
    return df
# usage_df = set_address_column(usage_df)
# usage_df

In [24]:
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)
period = int((end_date-start_date).days)+1

usage_df = analyze_total_usage(start_date, end_date, scale = "user")
# usage_df[usage] = usage_df[usage].apply(lambda x : int(x/period))
usage_df = usage_df.rename(columns = {geton_station_name: "STATION_NM", 
                          geton_station_latitude:"LOCAL_Y",
                          geton_station_longitude:"LOCAL_X",
                          usage:"USAGE"})

station_df = pd.read_csv("station_final.csv", encoding="CP949")

usage_key_columns = ["STATION_NM", "LOCAL_X", "LOCAL_Y"]
usage_df = set_address_column(usage_df)
usage_df.to_csv(make_new_path(start_date, end_date), encoding="CP949", index=False)

  8%|██████▎                                                                        | 288/3622 [00:21<04:33, 12.18it/s]

126.61324 33.2642


 25%|███████████████████▉                                                           | 912/3622 [01:08<03:12, 14.08it/s]

126.78144 33.30731
126.78142 33.30733
126.7814 33.30735


 34%|██████████████████████████▎                                                   | 1220/3622 [01:32<03:37, 11.04it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████▎| 3588/3622 [04:43<00:02, 12.05it/s]

126.61747 33.26697
126.61745 33.26699
126.61743 33.26701
126.61741 33.26703
126.61739 33.26705
126.61737 33.26706
126.61735 33.26708
126.61733 33.2671
126.61731 33.26712
126.61729 33.26714
126.61727 33.26716
126.61725 33.26717
126.61723 33.26719
126.61721 33.26721
126.61719 33.26723


 99%|█████████████████████████████████████████████████████████████████████████████▎| 3590/3622 [04:44<00:08,  3.84it/s]

126.61717 33.26725


100%|██████████████████████████████████████████████████████████████████████████████| 3622/3622 [04:46<00:00, 12.62it/s]
